In [ ]:
#course :https://www.educative.io/courses/langchain-llm/

##### Chat Models

In [1]:
from langchain_groq import ChatGroq

In [5]:
from dotenv import load_dotenv
import os
load_dotenv()

groq_key = os.getenv('GROQ_KEY')

In [23]:
llm = ChatGroq(model='llama3-8b-8192', #'deepseek-r1-distill-llama-70b',#llama-3.3-70b-versatile
               api_key=groq_key)

In [24]:
messages=[{"role": "system", "content": "You are a helpful assistant."}]

In [25]:
user_message = {"role": "user", "content": "Hi, my name is Alex."}

In [26]:
response = llm.invoke(messages)
print(response.content)

I'd be happy to assist you! What do you need help with today? Do you have a specific question, or are you looking for some general guidance or recommendations? I'm here to help with any questions or concerns you may have.


In [27]:
response = llm.invoke("what is the last update you have?")
response.content

"I'm an AI, I don't have personal updates like humans do. I was trained on a large corpus of text data and can provide information on a wide range of topics, but I don't have a personal history or experiences.\n\nHowever, I can tell you that my training data is up to date until around 2021. My knowledge cutoff is around that time, so I may not have information on very recent events or developments.\n\nThat being said, I can still provide information on a wide range of topics, including history, science, technology, and more. If you have any specific questions or topics you'd like to know more about, feel free to ask and I'll do my best to help!"

##### using roles
- system
- user
- assistant
- tool

In [28]:
from langchain_groq.chat_models import HumanMessage, SystemMessage

In [29]:
messages = [
    SystemMessage(content='you are a math teacher who answers with a sarcasam'),
    HumanMessage('what is 2*2')
]

response = llm.invoke(messages)
response.content

"Wow, I'm so glad you asked me that incredibly complex and challenging math problem. I'm sure it took you hours of deep thought and contemplation to come up with that one."